In [2]:
import numpy as np
import pymei as pm
import matplotlib.pyplot as plt


In [3]:
!python experiment0.py -h

usage: experiment0.py [-h] [-w WINDOW_SIZE] [-n NAME] [-l LOG] [-v]
                      [-c COOKIE_FILE] [-gt] [-tf TRAINING_FOLDER] [-cd]
                      [-d DATASET] [-cm] [-m MODEL] [-gtst]
                      [-tstf TEST_FOLDER] [-cl] [-o OUTPUT_IMAGE] -ne
                      NO_EVENT_PICKS -e EVENT_PICKS -i IMAGE

experiment0 involves the following steps:

    1. take an image (CMP stacked) of seismic amplitudes
    2. divide it up into windows of size w (w must be odd) centered at every possible pixel in the original image
    3. some of the windows for we which we have a class label
        ('event' or 'no event' depending on the classification of the central pixel)
        will be part of the training/validation set.
    4. all the other windows of the entire image will be part of the test set.
    5. we'll send POST requests to a running instance of an NVIDIA DIGITS server in order to:
        a) create a dataset
        b) create a model
        c) classify the te

In [7]:
import proj_utilities as proj

In [5]:
# makes sure a new window pops up
%matplotlib

Using matplotlib backend: Qt5Agg


## Load the data from file (DEPRECATED)

In [5]:
segy_traces = [trace for trace in pm.load('../inputs/sunset-stacked.sgy')]

In [6]:
segy_data_img = np.array([trace.data for trace in segy_traces]).T

In [7]:
vmax = np.abs(np.max(segy_data_img))
vmin = -vmax

In [11]:
no_event_training_points = {int(cdp): time for cdp, time in np.loadtxt('../inputs/original-noevent-points')}

In [12]:
no_event_training_points

{162: 595.89999999999998,
 169: 1378.4000000000001,
 179: 2080.6999999999998,
 191: 1085.5,
 200: 1631.2,
 212: 2241.1999999999998,
 226: 2534.0999999999999,
 235: 519.70000000000005,
 249: 2088.6999999999998,
 252: 1177.8,
 267: 1659.3,
 278: 531.70000000000005,
 280: 2594.3000000000002,
 301: 1378.4000000000001,
 314: 290.89999999999998,
 326: 1346.3,
 327: 2867.1999999999998,
 337: 973.10000000000002,
 341: 2586.3000000000002}

In [13]:
event_training_points = {int(cdp): time for cdp, time in np.loadtxt('../inputs/original-event-points')}

In [14]:
event_training_points

{157: 1022.5,
 164: 1809.8,
 198: 1025.2,
 209: 2199.0,
 210: 1917.5999999999999,
 218: 2180.3000000000002,
 233: 2161.5,
 245: 2156.0999999999999,
 248: 1022.5,
 254: 2165.0,
 255: 2022.8,
 274: 2206.3000000000002,
 286: 2246.3000000000002,
 288: 1019.8,
 302: 2313.0,
 307: 2146.8000000000002,
 331: 2206.0999999999999,
 346: 1022.5}

In [15]:
for trace in segy_traces: # have to run through since we don't really know if CDP indices are in order, what their starting index is, etc...
    # convert the time (in ms) found in the file to a sample (vertical axis) number
    if trace.cdp in no_event_training_points:
        no_event_training_points[trace.cdp] = int(no_event_training_points[trace.cdp] / 1e3 / trace.dt) 
    if trace.cdp in event_training_points:
        event_training_points[trace.cdp] = int(event_training_points[trace.cdp] / 1e3 / trace.dt) 


In [16]:
no_event_training_points

{162: 297,
 169: 689,
 179: 1040,
 191: 542,
 200: 815,
 212: 1120,
 226: 1267,
 235: 259,
 249: 1044,
 252: 588,
 267: 829,
 278: 265,
 280: 1297,
 301: 689,
 314: 145,
 326: 673,
 327: 1433,
 337: 486,
 341: 1293}

In [17]:
event_training_points

{157: 511,
 164: 904,
 198: 512,
 209: 1099,
 210: 958,
 218: 1090,
 233: 1080,
 245: 1078,
 248: 511,
 254: 1082,
 255: 1011,
 274: 1103,
 286: 1123,
 288: 509,
 302: 1156,
 307: 1073,
 331: 1103,
 346: 511}

In [18]:
cdp_shift = 48 # the CDP indices in the file start at 48 and progress until 48 + num_traces

no_event_time_indices = [no_event_training_points[k] for k in no_event_training_points.keys()]
no_event_cdp_indices = [k-cdp_shift for k in no_event_training_points.keys()]

event_time_indices = [event_training_points[k] for k in event_training_points.keys()]
event_cdp_indices = [k-cdp_shift for k in event_training_points.keys()]

## Fetch image and pick data from file

In [8]:
segy_traces, segy_data_img = proj.load_seismic_image('../inputs/sunset-stacked.sgy')

In [12]:
no_event_picks = proj.load_seismic_picks_json_format('../inputs/noevent-points.json')
event_picks = proj.load_seismic_picks_json_format('../inputs/event-points.json')

In [13]:
no_event_time_indices = [p[0] for p in no_event_picks]
no_event_cdp_indices = [p[1] for p in no_event_picks]

event_time_indices = [p[0] for p in event_picks]
event_cdp_indices = [p[1] for p in event_picks]

In [14]:
vmax = np.abs(np.max(segy_data_img))
vmin = -vmax

## Show sunset image with color scale normalized to <br> [-1.0, 1.0]

In [17]:
segy_data_img_normalized = segy_data_img / np.max(segy_data_img)

In [18]:
segy_data_img_normalized.min()

-0.29497638069238696

In [19]:
plt.imshow(segy_data_img_normalized, vmin=-1.0, vmax=1.0, cmap='seismic', aspect='auto')
plt.colorbar()
plt.xlabel('Common Midpoint Index')
plt.ylabel('Time Sample Index')
plt.title('synthetic "sunset" image')

## Show the absolute amplitudes of the image

In [20]:
segy_data_img_normalized_abs = np.abs(segy_data_img_normalized)

In [21]:
plt.imshow(segy_data_img_normalized, vmin=0.0, vmax=1.0, cmap='gray', aspect='auto')
plt.colorbar()
plt.xlabel('Common Midpoint Index')
plt.ylabel('Time Sample Index')
plt.title('synthetic "sunset" image absolute amplitudes')

## show the sunset image along with event and no event picks

In [16]:
plt.imshow(segy_data_img, cmap='seismic', vmin=vmin, vmax=vmax, aspect='auto')
plt.colorbar()

plt.scatter(no_event_cdp_indices, no_event_time_indices, c='b', marker='x', label='no event picks')
plt.scatter(event_cdp_indices, event_time_indices, c='r', marker='x', label='event picks')

plt.xlabel('Common Midpoint Index')
plt.ylabel('Time Sample Index')
plt.legend(loc='lower left')
plt.title('synthetic "sunset" image with event and non-event picks')

#plt.colorbar()

#plt.axis([0, segy_data_img.shape[1], 0, 500])

## show sunset image classified via the 17x17 window experiment (gray scale), as well as 33x33, 49x49, 65x65, 129x129

In [22]:
# the alpha channel that mpl baked into our saved images is mostly a nuisance
prob_window17 = plt.imread('/home/giuliano/quake-n-code/experiment0/outputs/experiment0--window17-corrected--2017-04-10--14-07-04.png')[:,:,:-1]
prob_window17version2 = plt.imread('/home/giuliano/quake-n-code/experiment0/outputs/experiment0--window17-corrected-version2--2017-04-17--10-51-38.png')[:,:,:-1]

prob_window33 = plt.imread('/home/giuliano/quake-n-code/experiment0/outputs/experiment0--window33-corrected--2017-04-03--13-41-43.png')[:,:,:-1]
prob_window49 = plt.imread('/home/giuliano/quake-n-code/experiment0/outputs/experiment0--window49-corrected--2017-04-03--17-56-47.png')[:,:,:-1]
prob_window65 = plt.imread('/home/giuliano/quake-n-code/experiment0/outputs/experiment0--window65-corrected--2017-04-03--17-58-25.png')[:,:,:-1]
prob_window129 = plt.imread('/home/giuliano/quake-n-code/experiment0/outputs/experiment0--window129-corrected-doneagain--2017-04-10--15-45-51.png')[:,:,:-1]


In [23]:
plt.imshow(prob_window17, aspect='auto', cmap='gray')
plt.colorbar()
plt.xlabel('Common Midpoint Index')
plt.ylabel('Time Sample Index')
plt.title('Synthetic sunset image classified with the probability of seismic event at each point (testing with 17x17 window size)')


In [24]:
plt.imshow(prob_window17version2, aspect='auto', cmap='gray')
plt.colorbar()
plt.xlabel('Common Midpoint Index')
plt.ylabel('Time Sample Index')
plt.title('Synthetic sunset image classified with the probability of seismic event at each point (testing with 17x17 window size)')


In [ ]:
# comparing 2 different runs of the same 17x17 experiment
plt.subplot(121)

plt.imshow(prob_window17, aspect='auto', cmap='gray')
plt.colorbar()
plt.xlabel('Common Midpoint Index')
plt.ylabel('Time Sample Index')
plt.title('Synthetic sunset image classified with the probability of seismic event at each point (testing with 17x17 window size)')

plt.subplot(122)

plt.imshow(prob_window17version2, aspect='auto', cmap='gray')
plt.colorbar()
plt.xlabel('Common Midpoint Index')
plt.ylabel('Time Sample Index')
plt.title('Synthetic sunset image classified with the probability of seismic event at each point (testing with 17x17 window size)')



In [ ]:
# comparing 2 different runs of the same 17x17 experiment
# thresholded at p1 and p2, respectively
p1 = 0.5
p2 = 0.3

plt.subplot(121)

plt.imshow(np.where(prob_window17 > p1,
                    1.0,
                    0.0),
           aspect='auto', cmap='gray')
plt.colorbar()
plt.xlabel('Common Midpoint Index')
plt.ylabel('Time Sample Index')
plt.title('Synthetic sunset image classified with the probability of seismic event at each point (testing with 17x17 window size)')

plt.subplot(122)

plt.imshow(np.where(prob_window17version2 > p2,
                    1.0,
                    0.0),
           aspect='auto', cmap='gray')
plt.colorbar()
plt.xlabel('Common Midpoint Index')
plt.ylabel('Time Sample Index')
plt.title('Synthetic sunset image classified with the probability of seismic event at each point (testing with 17x17 window size)')



In [ ]:
plt.imshow(prob_window33, aspect='auto', cmap='gray')
plt.colorbar()
plt.xlabel('Common Midpoint Index')
plt.ylabel('Time Sample Index')
plt.title('Synthetic sunset image classified with the probability of seismic event at each point (testing with 33x33 window size)')


In [ ]:
plt.imshow(prob_window49, aspect='auto', cmap='gray')
plt.colorbar()
plt.xlabel('Common Midpoint Index')
plt.ylabel('Time Sample Index')
plt.title('Synthetic sunset image classified with the probability of seismic event at each point (testing with 49x49 window size)')


In [ ]:
plt.imshow(prob_window65, aspect='auto', cmap='gray')
plt.colorbar()
plt.xlabel('Common Midpoint Index')
plt.ylabel('Time Sample Index')
plt.title('Synthetic sunset image classified with the probability of seismic event at each point (testing with 65x65 window size)')


In [ ]:
plt.imshow(prob_window129, aspect='auto', cmap='gray')
plt.colorbar()
plt.xlabel('Common Midpoint Index')
plt.ylabel('Time Sample Index')
plt.title('Synthetic sunset image classified with the probability of seismic event at each point (testing with 129x129 window size)')


In [ ]:
# generate a figure with all of them for comparison
plt.subplot(231)

plt.imshow(prob_window17, aspect='auto', cmap='gray')
plt.colorbar()
plt.xlabel('Common Midpoint Index')
plt.ylabel('Time Sample Index')
plt.title('(testing with 17x17 window size)')

plt.subplot(232)

plt.imshow(prob_window33, aspect='auto', cmap='gray')
plt.colorbar()
plt.xlabel('Common Midpoint Index')
plt.ylabel('Time Sample Index')
plt.title('(testing with 33x33 window size)')

plt.subplot(233)

plt.imshow(prob_window49, aspect='auto', cmap='gray')
plt.colorbar()
plt.xlabel('Common Midpoint Index')
plt.ylabel('Time Sample Index')
plt.title('(testing with 49x49 window size)')

plt.subplot(234)

plt.imshow(prob_window65, aspect='auto', cmap='gray')
plt.colorbar()
plt.xlabel('Common Midpoint Index')
plt.ylabel('Time Sample Index')
plt.title('(testing with 65x65 window size)')

plt.subplot(235)

plt.imshow(prob_window129, aspect='auto', cmap='gray')
plt.colorbar()
plt.xlabel('Common Midpoint Index')
plt.ylabel('Time Sample Index')
plt.title('(testing with 129x129 window size)')


## Threshold the probability at $p \in [0,1]$ for display

In [20]:
p = 0.2 # above this threshold: white; below: black
thresholded_prob_window17 = np.where(prob_window17version2 > p,
                                     1.0,
                                     0.0)

In [ ]:
# show the thresholded probability image along with the original image
# with absolute values of the amplitudes normalized at [0, 1]
plt.subplot(131)

plt.imshow(segy_data_img_normalized_abs, aspect='auto', cmap='gray')
plt.colorbar()
plt.xlabel('Common Midpoint Index')
plt.ylabel('Time Sample Index')
plt.title('original image with absolute normalized amplitudes')

plt.subplot(132)

plt.imshow(prob_window17version2, aspect='auto', cmap='gray')
plt.colorbar()
plt.xlabel('Common Midpoint Index')
plt.ylabel('Time Sample Index')
plt.title('estimated probabilities for 17x17 window size)')

plt.subplot(133)

plt.imshow(thresholded_prob_window17,  aspect='auto', cmap='gray')
plt.xlabel('Common Midpoint Index')
plt.ylabel('Time Sample Index')
plt.title('estimated event probability > ' + str(p))


## Inspect the histogram of the image - compare it to the probability map

In [21]:
segy_data_img_normalized_0_1 = (segy_data_img + np.abs(np.min(segy_data_img)))
segy_data_img_normalized_0_1 /= np.max(segy_data_img_normalized_0_1)

In [22]:
segy_data_img_ints = np.round(segy_data_img_normalized_0_1 * 255)

In [23]:
hist, bins = np.histogram(segy_data_img_ints, bins=256)

In [24]:
np.argmax(hist), np.max(hist)

(58, 1085412)

In [25]:
hist[np.argmax(hist)] = 0 # otherwise we can't see any of the others

In [ ]:
plt.bar(bins[:-1], hist)

## Color thresholding to select 'event' points from original image

In [32]:
threshold = 0.05
thresholded_segy_image = np.where(segy_data_img_normalized > threshold,
                                  1.0,
                                  0.0)

In [33]:
plt.subplot(121)

plt.imshow(segy_data_img_normalized_abs, vmin=-1.0, vmax=1.0, aspect='auto', cmap='seismic')
plt.colorbar()
plt.xlabel('Common Midpoint Index')
plt.ylabel('Time Sample Index')
plt.title('original image with absolute normalized amplitudes')

plt.subplot(122)

plt.imshow(thresholded_segy_image, vmin=-1.0, vmax=1.0, cmap='seismic', aspect='auto')
plt.colorbar()
plt.xlabel('Common Midpoint Index')
plt.ylabel('Time Sample Index')
plt.title('synthetic "sunset" image thresholded at ' + str(threshold))

In the range

## Edge detection to select 'event' points from original image

In [37]:
import cv2

In [38]:
img = np.uint8(segy_data_img_ints)

In [41]:
# play with the hysteresis thresholding parameters passed to Canny
edges = cv2.Canny(img, 50, 200)

In [42]:
plt.subplot(121)

plt.imshow(segy_data_img_ints, aspect='auto', cmap='gray')
plt.colorbar()
plt.xlabel('Common Midpoint Index')
plt.ylabel('Time Sample Index')
plt.title('original SGY image')

plt.subplot(122)

plt.imshow(edges, cmap='gray', aspect='auto')
plt.colorbar()
plt.xlabel('Common Midpoint Index')
plt.ylabel('Time Sample Index')
plt.title('original image subjected to Canny edge detection')


## Add some noise to the original image

In [ ]:
stddev_noise = 0.1
mean_noise = 0.0

segy_data_img_noisy = (segy_data_img_normalized 
                       + (np.random.standard_normal(
                           segy_data_img_normalized.shape)
                          * stddev_noise
                          + mean_noise))
                    

In [ ]:
plt.subplot(121)

plt.imshow(segy_data_img_normalized, aspect='auto', cmap='gray')
plt.colorbar()
plt.xlabel('Common Midpoint Index')
plt.ylabel('Time Sample Index')
plt.title('original normalized SGY image')

plt.subplot(122)

plt.imshow(segy_data_img_noisy, cmap='gray', aspect='auto')
plt.colorbar()
plt.xlabel('Common Midpoint Index')
plt.ylabel('Time Sample Index')
plt.title('original image with i.i.d Normal($\mu = ${}, $\sigma = ${}) noise added at each pixel'
         .format(mean_noise, stddev_noise))


In [ ]:
plt.imsave('/home/giuliano/quake-n-code/experiment0/inputs/noisy-sunset.png', segy_data_img_noisy, cmap='gray')

## Let's have a look at the results of classified noisy images

Noisy images were generated with the following command:

In [ ]:
!segyread tape='../inputs/sunset-stacked.sgy' | segyclean | suaddnoise -sn=0.5 > /tmp/example-noisy-sunset.su

## Fire up a Tkinter interface to play with some image parameters (probability display thresholds, for example)

In [ ]:
import PIL

In [ ]:
import tkinter as tk

In [ ]:
root = tk.Tk() # owns all the other widgets

In [ ]:
prob_window17.shape

In [ ]:
display_img = PIL.Image.fromarray(prob_window17, mode='F')

In [ ]:
#canv = tk.Canvas(root, width=800, height=600)
#canv.pack()




In [ ]:
photo = tk.PhotoImage()

In [ ]:
#canv.create_line(0, 0, 800, 600)
canv.create_image?

In [ ]:
tk.mainloop()

## Matplotlib figure animation example

In [ ]:
import matplotlib.animation as animation

In [ ]:
fig = plt.figure()


def f(x, y):
    return np.sin(x) + np.cos(y)

x = np.linspace(0, 2 * np.pi, 120)
y = np.linspace(0, 2 * np.pi, 100).reshape(-1, 1)

im = plt.imshow(f(x, y), cmap='gray', animated=True)


def updatefig(*args):
    global x, y
    x += np.pi / 15.
    y += np.pi / 20.
    im.set_array(f(x, y))
    return im,

ani = animation.FuncAnimation(fig, updatefig, interval=50, blit=True)
plt.show()